In [4]:
base_path = '' # your folder

import pandas as pd
train=pd.read_csv(base_path + 'train.csv')
test=pd.read_csv(base_path + 'test.csv')

In [5]:
train['ps_ind_0609_bin'] = train.apply(lambda x: 1 if x['ps_ind_06_bin'] == 1 else (2 if x['ps_ind_07_bin'] == 1 else 
(
3 if x['ps_ind_08_bin'] == 1 else (4 if x['ps_ind_09_bin'] == 1 else 5)

)), axis = 1)

test['ps_ind_0609_bin'] = test.apply(lambda x: 1 if x['ps_ind_06_bin'] == 1 else (2 if x['ps_ind_07_bin'] == 1 else 
(
3 if x['ps_ind_08_bin'] == 1 else (4 if x['ps_ind_09_bin'] == 1 else 5)

)), axis = 1)

train.drop(['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin'], axis = 1, inplace = True)

test.drop(['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin'], axis = 1, inplace = True)

train['ps_car_13'] = (train['ps_car_13']*train['ps_car_13']* 48400).round(0)

test['ps_car_13'] = (test['ps_car_13']*test['ps_car_13']* 48400).round(0)

train['ps_car_12'] = (train['ps_car_12']*train['ps_car_12']).round(4) * 10000

test['ps_car_12'] = (test['ps_car_12']*test['ps_car_12']).round(4) * 10000

for c in train[[c for c in train.columns if 'bin' in c]].columns:
    for cc in train[[c for c in train.columns if 'bin' in c]].columns:
            if train[train[cc] * train[c] == 0].shape[0] == train.shape[0]:
                print(c, cc)

train['ps_ind_161718_bin'] = train.apply(lambda x: 1 if x['ps_ind_16_bin'] == 1 else
                                        (2 if x['ps_ind_17_bin'] == 1 else 3), axis = 1
                                        )

test['ps_ind_161718_bin'] = test.apply(lambda x: 1 if x['ps_ind_16_bin'] == 1 else
                                        (2 if x['ps_ind_17_bin'] == 1 else 3), axis = 1
                                        )

train.drop(['ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin'], axis = 1, inplace = True)

test.drop(['ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin'], axis = 1, inplace = True)

train.to_csv(base_path + 'train_p.csv', index = False)

test.to_csv(base_path + 'test_p.csv', index = False)

ps_ind_16_bin ps_ind_17_bin
ps_ind_16_bin ps_ind_18_bin
ps_ind_17_bin ps_ind_16_bin
ps_ind_17_bin ps_ind_18_bin
ps_ind_18_bin ps_ind_16_bin
ps_ind_18_bin ps_ind_17_bin


In [10]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gc

print('loading files...')
train = pd.read_csv(base_path+'train_p.csv', na_values=-1)
test = pd.read_csv(base_path+'test_p.csv', na_values=-1)
col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(col_to_drop, axis=1)  
test = test.drop(col_to_drop, axis=1)  

for c in train.select_dtypes(include=['float64']).columns:
    train[c]=train[c].astype(np.float32)
    test[c]=test[c].astype(np.float32)
for c in train.select_dtypes(include=['int64']).columns[2:]:
    train[c]=train[c].astype(np.int8)
    test[c]=test[c].astype(np.int8)    

print(train.shape, test.shape)

# custom objective function (similar to auc)

def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)

def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

def gini_lgb(preds, dtrain):
    y = list(dtrain.get_label())
    score = gini(y, preds) / gini(y, y)
    return 'gini', score, True

# xgb
params = {'eta': 0.1, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 
          'objective': 'binary:logistic', 'eval_metric': 'auc', 'silent': False}

X = train.drop(['id', 'target'], axis=1)
features = X.columns
X = X.values
y = train['target'].values
sub=test['id'].to_frame()
sub['target']=0

sub_train = train['id'].to_frame()
sub_train['target']=0

nrounds= 200 #10**6  # need to change to 2000
kfold = 5  # need to change to 5
skf = StratifiedKFold(n_splits=kfold, random_state=0)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    d_train = xgb.DMatrix(X_train, y_train) 
    d_valid = xgb.DMatrix(X_valid, y_valid) 
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    xgb_model = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=10, 
                          feval=gini_xgb, maximize=True, verbose_eval=10)
    sub['target'] += xgb_model.predict(xgb.DMatrix(test[features].values), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (kfold)
    
    sub_train['target'] += xgb_model.predict(xgb.DMatrix(train[features].values), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (kfold)
    
gc.collect()
sub.head(2)

sub.to_csv(base_path+'test_sub_xgb.csv', index=False, float_format='%.5f')
sub_train.to_csv(base_path+'train_sub_xgb.csv', index=False, float_format='%.5f')

loading files...
(595212, 34) (892816, 33)
 xgb kfold: 1  of  5 : 
[0]	train-gini:0.198944	valid-gini:0.207771
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 10 rounds.
[10]	train-gini:0.242451	valid-gini:0.23631
[20]	train-gini:0.247742	valid-gini:0.237284
[30]	train-gini:0.258204	valid-gini:0.244316
[40]	train-gini:0.274173	valid-gini:0.256681
[50]	train-gini:0.287652	valid-gini:0.263266
[60]	train-gini:0.296775	valid-gini:0.2681
[70]	train-gini:0.304583	valid-gini:0.270837
[80]	train-gini:0.310406	valid-gini:0.273019
[90]	train-gini:0.315519	valid-gini:0.274764
[100]	train-gini:0.319895	valid-gini:0.276049
[110]	train-gini:0.323956	valid-gini:0.276949
[120]	train-gini:0.327344	valid-gini:0.277822
[130]	train-gini:0.330701	valid-gini:0.277986
Stopping. Best iteration:
[126]	train-gini:0.329063	valid-gini:0.278292

 xgb kfold: 2  of  5 : 
[0]	train-gini:0.199074	valid-gini:0.192664
Multiple eval met

In [11]:
# lgb
sub['target']=0
sub_train['target']=0

params = {'metric': 'auc', 'learning_rate' : 0.1, 'max_depth':8, 'max_bin':10,  'objective': 'binary', 
          'feature_fraction': 0.8,'bagging_fraction':0.9,'bagging_freq':5,  'min_data': 500}

skf = StratifiedKFold(n_splits=kfold, random_state=1)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' lgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]
    lgb_model = lgb.train(params, lgb.Dataset(X_train, label=y_train), nrounds, 
                  lgb.Dataset(X_eval, label=y_eval), verbose_eval=10, 
                  feval=gini_lgb, early_stopping_rounds=10)
    sub['target'] += lgb_model.predict(test[features].values, 
                        num_iteration=lgb_model.best_iteration) / (kfold)
    sub_train['target'] += lgb_model.predict(train[features].values, 
                        num_iteration=lgb_model.best_iteration) / (kfold)
    
sub.to_csv(base_path+'test_sub_lgb.csv', index=False, float_format='%.5f') 
sub_train.to_csv(base_path+'train_sub_lgb.csv', index=False, float_format='%.5f')

gc.collect()
sub.head(2)

 lgb kfold: 1  of  5 : 
Training until validation scores don't improve for 10 rounds.
[10]	valid_0's auc: 0.623345	valid_0's gini: 0.24667
[20]	valid_0's auc: 0.627064	valid_0's gini: 0.254126
[30]	valid_0's auc: 0.627838	valid_0's gini: 0.255675
[40]	valid_0's auc: 0.631762	valid_0's gini: 0.263524
[50]	valid_0's auc: 0.634841	valid_0's gini: 0.269682
[60]	valid_0's auc: 0.636169	valid_0's gini: 0.272339
[70]	valid_0's auc: 0.637842	valid_0's gini: 0.275685
[80]	valid_0's auc: 0.638769	valid_0's gini: 0.277538
Early stopping, best iteration is:
[79]	valid_0's auc: 0.638863	valid_0's gini: 0.277727
 lgb kfold: 2  of  5 : 
Training until validation scores don't improve for 10 rounds.
[10]	valid_0's auc: 0.621524	valid_0's gini: 0.243062
[20]	valid_0's auc: 0.626615	valid_0's gini: 0.25323
[30]	valid_0's auc: 0.629257	valid_0's gini: 0.258515
[40]	valid_0's auc: 0.634333	valid_0's gini: 0.268666
[50]	valid_0's auc: 0.637239	valid_0's gini: 0.274478
[60]	valid_0's auc: 0.639251	valid_0's 

,id,target
0,0,0.024575
1,1,0.027148


In [23]:
test[['id', 'target']].head()

,id,target
0,0,0.264810
1,1,0.360729
2,2,0.463984
3,3,0.026748
4,4,0.588131


In [33]:
# new_df = pd.concat([test[['id', 'target']], stack1[['target']].rename(columns = {'target' : 'stack'})], axis = 1)
# #new_df['final'] = np.mean([new_df['target'], new_df['stack']], axis = 1)
# new_df['final'] = (new_df['target'].rank() + new_df['stack'].rank())/(2 * test.shape[0])
# print(new_df.head())
# new_df.columns = ['id', 'new_stack', 'stack1', 'target']
# new_df[['id', 'target']].to_csv(base_path + 'rank_avg_lgbm_xgb_stack1.csv.gz', index = False, compression = 'gzip') 

   id    target     stack     final
0   0  0.264810  0.047145  0.325295
1   1  0.360729  0.044300  0.344407
2   2  0.463984  0.042049  0.372029
3   3  0.026748  0.031275  0.038791
4   4  0.588131  0.063401  0.607644


In [34]:
# import numpy as np
# import pandas as pd
# from sklearn import *
# from catboost import CatBoostClassifier
# from multiprocessing import *

# train = pd.read_csv(base_path + 'train_p.csv')
# test = pd.read_csv(base_path + 'test_p.csv')
# col = [c for c in train.columns if c not in ['id','target']]
# print(len(col))
# col = [c for c in col if not c.startswith('ps_calc_')]
# print(len(col))

# train = train.replace(-1, np.NaN)
# d_median = train.median(axis=0)
# d_mean = train.mean(axis=0)
# train = train.fillna(-1)

# def transform_df(df):
#     df = pd.DataFrame(df)
#     dcol = [c for c in df.columns if c not in ['id','target']]
#     df['ps_car_13_x_ps_reg_03'] = df['ps_car_13'] * df['ps_reg_03']
#     df['negative_one_vals'] = np.sum((df[dcol]==-1).values, axis=1)
#     for c in dcol:
#         if '_bin' not in c: #standard arithmetic
#             df[c+str('_median_range')] = (df[c].values > d_median[c]).astype(np.int)
#             df[c+str('_mean_range')] = (df[c].values > d_mean[c]).astype(np.int)
#             #df[c+str('_sq')] = np.power(df[c].values,2).astype(np.float32)
#             #df[c+str('_sqr')] = np.square(df[c].values).astype(np.float32)
#             #df[c+str('_log')] = np.log(np.abs(df[c].values) + 1)
#             #df[c+str('_exp')] = np.exp(df[c].values) - 1
    
#     return df

# def multi_transform(df):
#     print('Init Shape: ', df.shape)
#     #p = Pool(cpu_count())
#     df = p.map(transform_df, np.array_split(df, cpu_count()))
#     df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
#     #p.close(); p.join()
#     print('After Shape: ', df.shape)
#     return df

# def gini(y, pred):
#     fpr, tpr, thr = metrics.roc_curve(y, pred, pos_label=1)
#     g = 2 * metrics.auc(fpr, tpr) -1
#     return g

# def gini_catboost(pred, y):
#     return gini(y, pred)

# x1, x2, y1, y2 = model_selection.train_test_split(train, train['target'], test_size=0.25, random_state=99)

# x1 = transform_df(x1)
# x2 = transform_df(x2)
# test = transform_df(test)
# train = transform_df(train)

# col = [c for c in x1.columns if c not in ['id','target']]
# col = [c for c in col if not c.startswith('ps_calc_')]
# print(x1.values.shape, x2.values.shape)

# #remove duplicates just in case
# #tdups = transform_df(train)
# #dups = tdups[tdups.duplicated(subset=col, keep=False)]

# #x1 = x1[~(x1['id'].isin(dups['id'].values))]
# #x2 = x2[~(x2['id'].isin(dups['id'].values))]
# #print(x1.values.shape, x2.values.shape)

# y1 = x1['target']
# y2 = x2['target']
# x1 = x1[col]
# x2 = x2[col]

# model3 = CatBoostClassifier(iterations=200, learning_rate=0.1, depth=7, loss_function='Logloss', eval_metric='AUC', random_seed=0, od_type='Iter', od_wait=100) 
# model3.fit(x1[col], y1, eval_set=(x2[col], y2), use_best_model=True, verbose=True, )
# print(gini_catboost(model3.predict_proba(x2[col])[:,1], y2))
# test['target'] = model3.predict_proba(test[col])[:,1]
# test['target'] = (np.exp(test['target'].values) - 1.0).clip(0,1)
# train['target'] = model3.predict_proba(train[col])[:,1]
# train['target'] = (np.exp(train['target'].values) - 1.0).clip(0,1)
# test[['id','target']].to_csv(base_path + 'test_catboost_submission.csv', index=False, float_format='%.5f')
# train[['id','target']].to_csv(base_path + 'train_catboost_submission.csv', index=False, float_format='%.5f')



/Users/neerjadoshi/anaconda2/envs/fastai/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/neerjadoshi/anaconda2/envs/fastai/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)
/Users/neerjadoshi/anaconda2/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module '_catboost' does not match runtime version 3.6
  return f(*args, **kwds)


52
32
(446409, 136) (148803, 136)
Borders for float features generated
0:	learn 0.547204263	test 0.5446748932	bestTest 0.5446748932		total: 950ms	remaining: 3m 8s
1:	learn 0.573248505	test 0.5731519818	bestTest 0.5731519818		total: 1.87s	remaining: 3m 4s
2:	learn 0.5915055485	test 0.592701179	bestTest 0.592701179		total: 2.74s	remaining: 3m
3:	learn 0.6052177179	test 0.6026552012	bestTest 0.6026552012		total: 3.67s	remaining: 2m 59s
4:	learn 0.611776931	test 0.6084025222	bestTest 0.6084025222		total: 4.61s	remaining: 2m 59s
5:	learn 0.6127940259	test 0.6102924295	bestTest 0.6102924295		total: 5.52s	remaining: 2m 58s
6:	learn 0.6124746553	test 0.6101776241	bestTest 0.6102924295		total: 6.17s	remaining: 2m 50s
7:	learn 0.6120952734	test 0.6101082714	bestTest 0.6102924295		total: 7.06s	remaining: 2m 49s
8:	learn 0.6130767979	test 0.6103475635	bestTest 0.6103475635		total: 7.8s	remaining: 2m 45s
9:	learn 0.6139701019	test 0.6113433246	bestTest 0.6113433246		total: 8.72s	remaining: 2m 45s
1

87:	learn 0.6515920553	test 0.6355646206	bestTest 0.6355646206		total: 1m 18s	remaining: 1m 39s
88:	learn 0.6518456379	test 0.6356922855	bestTest 0.6356922855		total: 1m 19s	remaining: 1m 38s
89:	learn 0.6521337232	test 0.6356939644	bestTest 0.6356939644		total: 1m 20s	remaining: 1m 37s
90:	learn 0.6525042603	test 0.635877195	bestTest 0.635877195		total: 1m 20s	remaining: 1m 36s
91:	learn 0.6529836174	test 0.6360302948	bestTest 0.6360302948		total: 1m 21s	remaining: 1m 36s
92:	learn 0.6532447353	test 0.6359851824	bestTest 0.6360302948		total: 1m 22s	remaining: 1m 35s
93:	learn 0.6534606192	test 0.6360043257	bestTest 0.6360302948		total: 1m 23s	remaining: 1m 34s
94:	learn 0.6535840837	test 0.6360434901	bestTest 0.6360434901		total: 1m 24s	remaining: 1m 33s
95:	learn 0.6538178923	test 0.6359832287	bestTest 0.6360434901		total: 1m 25s	remaining: 1m 32s
96:	learn 0.6539687216	test 0.6358772676	bestTest 0.6360434901		total: 1m 26s	remaining: 1m 31s
97:	learn 0.6542391028	test 0.6359444053	b

173:	learn 0.6676710839	test 0.6371740322	bestTest 0.637515879		total: 2m 35s	remaining: 23.2s
174:	learn 0.6677267572	test 0.6371524321	bestTest 0.637515879		total: 2m 36s	remaining: 22.3s
175:	learn 0.667887388	test 0.6372306376	bestTest 0.637515879		total: 2m 37s	remaining: 21.4s
176:	learn 0.6682406453	test 0.6372366972	bestTest 0.637515879		total: 2m 38s	remaining: 20.5s
177:	learn 0.6685133478	test 0.6372469767	bestTest 0.637515879		total: 2m 38s	remaining: 19.6s
178:	learn 0.6686699018	test 0.6372100164	bestTest 0.637515879		total: 2m 39s	remaining: 18.8s
179:	learn 0.6687907129	test 0.6372092126	bestTest 0.637515879		total: 2m 40s	remaining: 17.9s
180:	learn 0.6688416176	test 0.6371799272	bestTest 0.637515879		total: 2m 41s	remaining: 17s
181:	learn 0.6689610741	test 0.6372016478	bestTest 0.637515879		total: 2m 42s	remaining: 16.1s
182:	learn 0.6691249772	test 0.6372129023	bestTest 0.637515879		total: 2m 43s	remaining: 15.2s
183:	learn 0.6694203218	test 0.6371998885	bestTest 0.

In [46]:

# test files
stack1 = pd.read_csv('stacked_1.csv')
test_xgb = pd.read_csv(base_path + 'test_sub_xgb.csv')
test_lgb = pd.read_csv(base_path + 'test_sub_lgb.csv')
# test_dnn = pd.read_csv(base_path + 'test_dnn_predictions.csv')
# test_up = pd.read_csv(base_path + 'test_submission.csv')
#test_cat = pd.read_csv(base_path + 'test_catboost_submission.csv')
# test_kin = pd.read_csv(base_path + 'test_uberKinetics.csv')
# test_gp = pd.read_csv(base_path + 'test_gpari.csv')

test=pd.read_csv(base_path + 'test.csv')


test = pd.concat([test, 
                   test_xgb[['target']].rename(columns = {'target' : 'xgb'}),
                   test_lgb[['target']].rename(columns = {'target' : 'lgb'}),
#                    test_dnn[['target']].rename(columns = {'target' : 'dnn'}),
#                    test_up[['target']].rename(columns = {'target' : 'up'}),
#                 test_cat[['target']].rename(columns = {'target' : 'cat'}),
#                    test_kin[['target']].rename(columns = {'target' : 'kin'}),
#                    test_gp[['target']].rename(columns = {'target' : 'gp'})
                  stack1[['target']].rename(columns = {'target' : 'stack'})
                  ], axis = 1)

train_cols = ['xgb', 'lgb', 'stack']
#train_cols = ['xgb', 'lgb', 'dnn', 'up', 'cat', 'kin', 'gp']

for t in train_cols:
    test[t + '_rank'] = test[t].rank()

for t in train_cols:
    test[t + '_rank'] = test[t].rank()
print(len(test['xgb']))
print(len(test['lgb']))
print(len(test['stack']))

test['target'] = test[['xgb', 'lgb', 'stack']].mean(axis=1)
#test['target'] = (test['xgb_rank'] + test['lgb_rank'] +test['stack_rank'])/(3 * test.shape[0])
#test['target'] = (test['xgb_rank'] + test['lgb_rank'] + test['dnn_rank'] + test['up_rank'] + \
#                  test['cat_rank'] + test['kin_rank'] + test['gp_rank']) / (7 * test.shape[0])
test[['id', 'target']].to_csv(base_path + 'rank_avg_lgbm_xgb_stack1_mean.csv.gz', index = False, compression = 'gzip') 

892816
892816
892816


0         0.037978
1         0.038960
2         0.040280
3         0.026478
4         0.050264
5         0.061672
6         0.028356
7         0.049340
8         0.072645
9         0.081197
10        0.042372
11        0.034988
12        0.058952
13        0.061919
14        0.064996
15        0.032561
16        0.036719
17        0.076335
18        0.025975
19        0.081280
20        0.050651
21        0.075079
22        0.081409
23        0.027450
24        0.037788
25        0.037604
26        0.142415
27        0.059298
28        0.037157
29        0.028136
            ...   
892786    0.028708
892787    0.046325
892788    0.047847
892789    0.048604
892790    0.048138
892791    0.039376
892792    0.039212
892793    0.055495
892794    0.036357
892795    0.048241
892796    0.082164
892797    0.098439
892798    0.052594
892799    0.151616
892800    0.038086
892801    0.040711
892802    0.040772
892803    0.044280
892804    0.066957
892805    0.045834
892806    0.037194
892807    0.

150    0.188827
151    0.027185
152    0.053618
153    0.028737
154    0.026434
155    0.070561
156    0.075738
157    0.043073
158    0.038377
159    0.044140
160    0.121006
161    0.035458
162    0.042560
163    0.033056
164    0.044098
165    0.024820
166    0.036434
167    0.036668
168    0.024154
169    0.086391
170    0.045251
171    0.071042
172    0.079558
173    0.073117
174    0.047517
175    0.050846
176    0.031933
177    0.038984
178    0.100594
179    0.042305
180    0.078455
181    0.040199
182    0.092254
183    0.041887
184    0.044591
185    0.040523
186    0.144497
187    0.060536
188    0.047316
189    0.041010
190    0.101201
191    0.038987
192    0.305332
193    0.041176
194    0.089928
195    0.027068
196    0.053228
197    0.053064
198    0.140266
199    0.048396
dtype: float64